# Análisis de Municipios en España para Determinar el Mejor Lugar para Vivir

## Introducción

El objetivo de este análisis es determinar cuáles son los mejores municipios en España para vivir, teniendo en cuenta varios factores clave:

1. **Renta Neta Media por Municipio**: Evaluar la capacidad económica de los residentes.
2. **Gastos en Comida**: Estimar el coste de vida en términos de alimentación.
3. **Precio de Alquiler**: Analizar el coste del alquiler de viviendas.
4. **Número de Empresas por Municipio**: Considerar las oportunidades laborales disponibles.

## Análisis y Resultados

Una vez que los datos estén limpios, se procederá con el análisis para identificar los municipios que ofrecen las mejores condiciones para vivir según los criterios mencionados. Se calcularán indicadores clave y se compararán los municipios para tomar una decisión informada.

## Conclusión

El análisis proporcionará una visión clara de cuáles son los mejores municipios en España para vivir, teniendo en cuenta la renta neta, el coste de vida y las oportunidades laborales.

---

**Nota**: Este notebook está diseñado para ser ejecutado en un entorno local o en una plataforma de notebooks como Jupyter. Asegúrate de ajustar las rutas de los archivos según sea necesario.


## Preprocesamiento de Datos - Archivo "precio alquiler compartido"

Dado que los datos pueden contener valores faltantes (NaN), el primer paso será preprocesar los datos para asegurar su calidad. En este notebook, se realizarán las siguientes tareas de preprocesamiento:

1. **Lectura de Datos**: Cargar los archivos CSV que contienen la información necesaria.
2. **Relleno de Valores Faltantes**: Para los datos faltantes en la columna de precio de alquiler compartido, se calculará la media de los valores disponibles para el mismo municipio y se rellenarán los valores faltantes con esta media.
3. **Guardado de Datos Limpios**: Guardar los datos preprocesados en nuevos archivos CSV para su uso posterior. (Se mantiene el original para comprobar el correcto funcionamiento y tenerlo como backup para saber si se ha cometido alguna errata en el preprocesado)

In [3]:
import pandas as pd

# Leer el archivo CSV
file_path = 'precio alquiler compartido.csv'
data = pd.read_csv(file_path, delimiter=';')

# Mostrar las primeras filas del DataFrame para verificar la carga de datos
print(data.head())

# Calcular la media de los valores disponibles para cada municipio (NPRO)
mean_values = data.groupby('NPRO')['ALQTBID12_M_VC_11'].transform('mean')

# Rellenar los valores faltantes con la media calculada
data['ALQTBID12_M_VC_11'].fillna(mean_values, inplace=True)

# Mostrar las primeras filas del DataFrame después de rellenar los valores faltantes
print(data.head())

# Guardar el DataFrame modificado en un nuevo archivo CSV
output_file_path = 'Preprocesados/precio_alquiler_compartido_preprocesado.csv'
data.to_csv(output_file_path, index=False, sep=';')

print("Proceso completado. El archivo limpio ha sido guardado en:", output_file_path)


       NPRO  CUMUN       NMUN  ALQTBID12_M_VC_11
0  Albacete   2001  Abengibre                NaN
1  Albacete   2002     Alatoz                NaN
2  Albacete   2003   Albacete              450.0
3  Albacete   2004   Albatana                NaN
4  Albacete   2005    Alborea                NaN
       NPRO  CUMUN       NMUN  ALQTBID12_M_VC_11
0  Albacete   2001  Abengibre         282.571429
1  Albacete   2002     Alatoz         282.571429
2  Albacete   2003   Albacete         450.000000
3  Albacete   2004   Albatana         282.571429
4  Albacete   2005    Alborea         282.571429
Proceso completado. El archivo limpio ha sido guardado en: Preprocesados/precio_alquiler_compartido_preprocesado.csv


## Preprocesamiento de Datos - Archivo "empresas por municipio"

Dado que los datos pueden contener valores faltantes (en este caso vienen rellenados con "."), el primer paso será preprocesar los datos para asegurar su calidad. En este notebook, se realizarán las siguientes tareas de preprocesamiento:

1. **Lectura de Datos**: Cargar los archivos CSV que contienen la información necesaria.
2. **Relleno de Valores Faltantes**: Para los datos faltantes en la columna "Total" correspondiente al Nº total de empresas por municipio, 1º se sustitullen los valores nulos, en este caso dados en "." por NaN para manejar mejor (que no quiere decir que sean 0 sino que no se dispone del valor). 2º Se han rellenado los valores faltantes con interpolacion lineal para los casos en los que existia un historico. 3º Se ha realizado un "bfill" y un "ffill" porque en algunos casos la interpolación dejaba los datos de las esquinas (primer y ultimo dato) vacios. 4º Se ha realizado un redondeo porque no existen medias empresas. 
3. **Guardado de Datos Limpios**: Guardar los datos preprocesados en nuevos archivos CSV para su uso posterior. (Se mantiene el original para comprobar el correcto funcionamiento y tenerlo como backup para saber si se ha cometido alguna errata en el preprocesado)

In [14]:
import pandas as pd
import numpy as np

# Leer el archivo CSV
file_path = 'empresas por municipio.csv'
data = pd.read_csv(file_path, delimiter=';')

# Mostrar las primeras filas del DataFrame para verificar la carga de datos
print(data.head())

# Reemplazar los valores "." por NaN para facilitar el procesamiento
data.replace('.', np.nan, inplace=True)

# Convertir la columna 'Total' a tipo numérico
data['Total'] = pd.to_numeric(data['Total'], errors='coerce')

# Función para rellenar valores faltantes utilizando interpolación lineal y redondeo
def fill_missing_values(group):
    # Interpolación lineal para rellenar valores faltantes
    group['Total'] = group['Total'].interpolate(method='linear', limit_direction='both')
    # Rellenar valores faltantes al principio y al final con el valor más cercano disponible
    group['Total'] = group['Total'].fillna(method='bfill').fillna(method='ffill')
    # Redondear los valores interpolados y convertir a entero
    group['Total'] = group['Total'].round()
    return group

# Aplicar la función a cada grupo de municipios
data = data.groupby('Municipios').apply(fill_missing_values)

# Mostrar las primeras filas del DataFrame después de rellenar los valores faltantes
print(data.head())

# Guardar el DataFrame modificado en un nuevo archivo CSV
output_file_path = 'Preprocesados/empresas_por_municipio_limpio.csv'
data.to_csv(output_file_path, index=False, sep=';')

print("Proceso completado. El archivo limpio ha sido guardado en:", output_file_path)


               Municipios  Periodo Total
0  01001 Alegría-Dulantzi     2024   148
1  01001 Alegría-Dulantzi     2023   150
2  01001 Alegría-Dulantzi     2022   154
3  01001 Alegría-Dulantzi     2021   169
4  01001 Alegría-Dulantzi     2020   155
                                      Municipios  Periodo  Total
Municipios                                                      
01001 Alegría-Dulantzi 0  01001 Alegría-Dulantzi     2024  148.0
                       1  01001 Alegría-Dulantzi     2023  150.0
                       2  01001 Alegría-Dulantzi     2022  154.0
                       3  01001 Alegría-Dulantzi     2021  169.0
                       4  01001 Alegría-Dulantzi     2020  155.0
Proceso completado. El archivo limpio ha sido guardado en: Preprocesados/empresas_por_municipio_limpio.csv


## Preprocesamiento de Datos - Archivo "renta neta municipios"

Dado que los datos pueden contener valores faltantes (en este caso vienen rellenados con "."), el primer paso será preprocesar los datos para asegurar su calidad. En este notebook, se realizarán las siguientes tareas de preprocesamiento:

1. **Lectura de Datos**: Cargar los archivos CSV que contienen la información necesaria.
2. **Relleno de Valores Faltantes**: Para los datos faltantes en la columna "Total" correspondiente a la renta per cápita por municipio, 1º se sustitullen los valores nulos, en este caso dados en "." por NaN para manejar mejor (que no quiere decir que sean 0 sino que no se dispone del valor). 2º Se han rellenado los valores faltantes con interpolacion lineal para los casos en los que existia un historico. 3º Se ha realizado un "bfill" y un "ffill" porque en algunos casos la interpolación dejaba los datos de las esquinas (primer y ultimo dato) vacios. 4º Se ha realizado un redondeo a dos decimales porque por debajo de los centimos no existe valor. 
3. **Guardado de Datos Limpios**: Guardar los datos preprocesados en nuevos archivos CSV para su uso posterior. (Se mantiene el original para comprobar el correcto funcionamiento y tenerlo como backup para saber si se ha cometido alguna errata en el preprocesado)

In [19]:
import pandas as pd
import numpy as np

# Leer el archivo CSV
file_path = 'renta neta municipios.csv'
data = pd.read_csv(file_path, delimiter=';')

# Mostrar las primeras filas del DataFrame para verificar la carga de datos
print(data.head())

# Reemplazar los valores "." por NaN para facilitar el procesamiento
data.replace('.', np.nan, inplace=True)

# Función para convertir valores en formato español a numérico
def convert_to_numeric(value):
    if pd.isna(value):
        return np.nan
    return float(value.replace('.', '').replace(',', '.'))

# Aplicar la función a la columna 'Total'
data['Total'] = data['Total'].apply(convert_to_numeric)

# Función para rellenar valores faltantes utilizando interpolación lineal
def fill_missing_values(group):
    # Interpolación lineal para rellenar valores faltantes
    group['Total'] = group['Total'].interpolate(method='linear', limit_direction='both')
    # Rellenar valores faltantes al principio y al final con el valor más cercano disponible
    group['Total'] = group['Total'].fillna(method='bfill').fillna(method='ffill')
    # Redondear los valores interpolados
    group['Total'] = group['Total'].round(2)
    return group

# Aplicar la función a cada grupo de municipios
data = data.groupby('Municipios').apply(fill_missing_values)

# Mostrar las primeras filas del DataFrame después de rellenar los valores faltantes
print(data.head())

# Guardar el DataFrame modificado en un nuevo archivo CSV
output_file_path = 'Preprocesados/renta_neta_municipios_limpio.csv'
data.to_csv(output_file_path, index=False, sep=';')

print("Proceso completado. El archivo limpio ha sido guardado en:", output_file_path)


        Municipios  Periodo     Total
0  02001 Abengibre     2022  13.063,0
1  02001 Abengibre     2021  12.188,0
2  02001 Abengibre     2020  11.675,0
3  02001 Abengibre     2019  11.132,0
4  02001 Abengibre     2018  10.854,0
                                         Municipios  Periodo    Total
Municipios                                                           
01001 Alegría-Dulantzi 2656  01001 Alegría-Dulantzi     2022  15116.0
                       2657  01001 Alegría-Dulantzi     2021  14647.0
                       2658  01001 Alegría-Dulantzi     2020  13969.0
                       2659  01001 Alegría-Dulantzi     2019  14299.0
                       2660  01001 Alegría-Dulantzi     2018  13361.0
Proceso completado. El archivo limpio ha sido guardado en: Preprocesados/renta_neta_municipios_limpio.csv
